## Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from collections import namedtuple

%matplotlib inline

In [2]:
NUM_GAMES = 10000
NUM_ROUNDS = 100

results = []

In [3]:
NUM_PROPERTIES = 40
GO = 0
GO_TO_JAIL = 30
JAIL = 10
BOARDWALK = 39
READING = 5
ILLINOIS = 25
ST_CHARLES = 11
CHANCE = (7, 22, 36)
COMMUNITY_CHEST = (2, 17, 33)
UTILITIES = (12, 28)
RAILROADS = (5, 15, 25, 35)

In [4]:
Property = namedtuple('Property', ['name', 'color', 'low_rent', 'high_rent', 'cost', 'full_cost'])

In [5]:
properties = [
    Property('Go', 'None', 0, 0, 1, 1),  # Non-purchasable have cost of 1 to prevent div-zero errors later.
    Property('Mediterranean Avenue', 'Dark Purple', 2, 250, 60, 310),
    Property('Community Chest', 'None', 0, 0, 1, 1),
    Property('Baltic Avenue', 'Dark Purple', 4, 320, 60, 310),
    Property('Income Tax', 'None', 0, 0, 1, 1),
    Property('Reading Railroad', 'Railroad', 25, 200, 200, 200),
    Property('Oriental Avenue', 'Light Blue', 6, 550, 100, 350),
    Property('Chance', 'None', 0, 0, 1, 1),
    Property('Vermont Avenue', 'Light Blue', 6, 550, 100, 350),
    Property('Connecticut Avenue', 'Light Blue', 8, 600, 120, 370),
    Property('Jail', 'None', 0, 0, 1, 1),
    Property('St. Charles Place', 'Pink', 10, 750, 140, 640),
    Property('Electric Company', 'Utility', 28, 70, 150, 150),  # Rent based on mean roll of 7.
    Property('States Avenue', 'Pink', 10, 750, 140, 640),
    Property('Virginia Avenue', 'Pink', 12, 900, 160, 660),
    Property('Pennsylvania Railroad', 'Railroad', 25, 200, 200, 200),
    Property('St. James Place', 'Orange', 14, 950, 180, 680),
    Property('Community Chest', 'None', 0, 0, 1, 1),
    Property('Tennessee Avenue', 'Orange', 14, 950, 180, 680),
    Property('New York Avenue', 'Orange', 16, 1000, 200, 700),
    Property('Free Parking', 'None', 0, 0, 1, 1),
    Property('Kentucky Avenue', 'Red', 18, 1050, 220, 970),
    Property('Chance', 'None', 0, 0, 1, 1),
    Property('Indiana Avenue', 'Red', 18, 1050, 220, 970),
    Property('Illinois Avenue', 'Red', 20, 1100, 240, 990),
    Property('B&O Railroad', 'Railroad', 25, 200, 200, 200),
    Property('Atlantic Avenue', 'Yellow', 22, 1150, 260, 1010),
    Property('Ventnor Avenue', 'Yellow', 22, 1150, 260, 1010),
    Property('Water Works', 'Utility', 28, 70, 150, 150),  # Rent based on mean roll of 7.
    Property('Marvin Garden', 'Yellow', 24, 1200, 280, 1030),
    Property('Go To Jail', 'None', 0, 0, 1, 1),
    Property('Pacific Avenue', 'Green', 26, 1275, 300, 1300),
    Property('North Carolina Avenue', 'Green', 26, 1275, 300, 1300),
    Property('Community Chest', 'None', 0, 0, 1, 1),
    Property('Pennsylvania Avenue', 'Green', 28, 1400, 320, 1320),
    Property('Short Line', 'Railroad', 25, 200, 200, 200), 
    Property('Chance', 'None', 0, 0, 1, 1),
    Property('Park Place', 'Blue', 35, 1500, 350, 1350),
    Property('Luxury Tax', 'None', 0, 0, 1, 1),
    Property('Boardwalk', 'Blue', 50, 2000, 400, 1400)
]

In [6]:
class Player():
    
    def __init__(self, number, game):
        self.number = number
        self.game = game
        self.position = 0
        self.turn = 0
        self.round = 1
    
    def roll_dice(self):
        dice_one = random.randint(1, 6)
        dice_two = random.randint(1, 6)

        return dice_one + dice_two, dice_one == dice_two
    
    def go_to_property(self, number, pass_go=True):
        if pass_go and self.position > number:
            self.round += 1
        
        self.position = number
        
        results.append([self.game.number, 
                        self.number, 
                        self.turn, 
                        0, 
                        False, 
                        self.position, 
                        properties[self.position].name])
        
        self.resolve_property()

    def take_turn(self):
        global df
        
        self.turn += 1
        doubles_count = 0
        
        while True:
            roll, doubles = self.roll_dice()
            
            if doubles:
                doubles_count += 1
            
            if doubles_count >= 3:
                self.go_to_property(JAIL, pass_go=False)
                break
            
            self.position += roll
            
            if self.position >= NUM_PROPERTIES:
                self.position = self.position % NUM_PROPERTIES
                self.round += 1
            
            results.append([self.game.number, 
                            self.number, 
                            self.turn, 
                            roll, 
                            doubles, 
                            self.position, 
                            properties[self.position].name])
            
            end_turn = self.resolve_property()
            
            if end_turn:
                break
                    
            if not doubles:
                break
    
    def resolve_property(self):
        
        if self.position == GO_TO_JAIL:
            self.go_to_property(JAIL, pass_go=False)
            return True
            
        elif self.position in CHANCE:
            card = self.game.draw_chance()

            if card == 0:  # Advance to Go
                self.go_to_property(GO)
            elif card == 1:  # Go to Jail
                self.go_to_property(JAIL, pass_go=False)
                return True
            elif card == 2:  # Go Back 3 Spaces
                new_position = self.position - 3
                if new_position < 0:
                    new_position += NUM_PROPERTIES
                self.go_to_property(new_position)
            elif card == 3:  # Advance to Boardwalk
                self.go_to_property(BOARDWALK)
            elif card == 4:  # Advance to Reading Railroad
                self.go_to_property(READING)
            elif card == 5:  # Advance to Illinois Avenue
                self.go_to_property(ILLINOIS)
            elif card == 6:  # Advance to St. Charles Place
                self.go_to_property(ST_CHARLES)
            elif card == 7:  # Advance to nearest Utility
                found = False
                for util in UTILITIES:
                    if self.position < util:
                        found = True
                        self.go_to_property(util)
                
                if not found:
                    self.go_to_property(UTILITIES[0])
            elif card == 8 or card == 9:
                found = False
                for rr in RAILROADS:
                    if self.position < rr:
                        found = True
                        self.go_to_property(rr)
                
                if not found:
                    self.go_to_property(RAILROADS[0])
                    

        elif self.position in COMMUNITY_CHEST:
            card = self.game.draw_community_chest()

            if card == 0:  # Advance to Go
                self.go_to_property(GO)
            elif card == 1:  # Go to Jail
                self.go_to_property(JAIL, pass_go=False)
                return True
        
        return False

In [7]:
class Game():
    
    def __init__(self, number, num_players):
        self.number = number
        self.players = [Player(i + 1, self) for i in range(num_players)]
        self.chance = [i for i in range(16)]
        random.shuffle(self.chance)
        self.community_chest = [i for i in range(16)]
        random.shuffle(self.community_chest)
    
    def play_game(self, num_turns):
        for i in range(num_turns):
            for p in self.players:
                p.take_turn()
    
    def draw_card(self, deck):
        card = deck.pop(0)
        deck.append(card)
        return card
    
    def draw_community_chest(self):
        return self.draw_card(self.community_chest)

    def draw_chance(self):
        return self.draw_card(self.chance)

## Determine Return on Investment Rate

In [8]:
investments = pd.DataFrame(properties, columns=['Name', 'Color', 'Low Rent', 'High Rent', 'Low Cost', 'High Cost'])

In [9]:
investments.head()

,Name,Color,Low Rent,High Rent,Low Cost,High Cost
0,Go,None,0,0,1,1
1,Mediterranean Avenue,Dark Purple,2,250,60,310
2,Community Chest,None,0,0,1,1
3,Baltic Avenue,Dark Purple,4,320,60,310
4,Income Tax,None,0,0,1,1


In [10]:
investments['Low ROI'] = investments['Low Cost'] / investments['Low Rent']
investments['High ROI'] = investments['High Cost'] / investments['High Rent']

In [13]:
columns = ['Name', 'Low Cost', 'Low Rent', 'Low ROI']
investments.sort_values('Low ROI')[columns]

,Name,Low Cost,Low Rent,Low ROI
28,Water Works,150,28,5.357143
12,Electric Company,150,28,5.357143
25,B&O Railroad,200,25,8.000000
15,Pennsylvania Railroad,200,25,8.000000
5,Reading Railroad,200,25,8.000000
39,Boardwalk,400,50,8.000000
35,Short Line,200,25,8.000000
37,Park Place,350,35,10.000000
34,Pennsylvania Avenue,320,28,11.428571
31,Pacific Avenue,300,26,11.538462


In [14]:
columns = ['Name', 'High Cost', 'High Rent', 'High ROI']
investments.sort_values('High ROI')[columns]

,Name,High Cost,High Rent,High ROI
9,Connecticut Avenue,370,600,0.616667
8,Vermont Avenue,350,550,0.636364
6,Oriental Avenue,350,550,0.636364
19,New York Avenue,700,1000,0.700000
39,Boardwalk,1400,2000,0.700000
18,Tennessee Avenue,680,950,0.715789
16,St. James Place,680,950,0.715789
14,Virginia Avenue,660,900,0.733333
11,St. Charles Place,640,750,0.853333
13,States Avenue,640,750,0.853333


## Run the Monte Carlo Simulation

In [15]:
for i in range(NUM_GAMES):
    g = Game(i, 4)
    g.play_game(NUM_ROUNDS)

In [16]:
game_results = pd.DataFrame(results, columns=['Game', 'Player', 'Turn', 'Roll', 'Doubles', 'Position', 'Property'])

In [17]:
game_results.head()

,Game,Player,Turn,Roll,Doubles,Position,Property
0,0,1,1,10,False,10,Jail
1,0,2,1,3,False,3,Baltic Avenue
2,0,3,1,11,False,11,St. Charles Place
3,0,4,1,5,False,5,Reading Railroad
4,0,1,2,3,False,13,States Avenue


## Count total hits per property

In [18]:
prop_revenue = pd.DataFrame({'Count' : game_results.groupby( ['Position', 'Property'] ).size()}).reset_index()
prop_revenue['Color'] = prop_revenue['Position'].apply(lambda x: properties[x].color)
prop_revenue['Percent Total Count'] = prop_revenue['Count'].apply(lambda x: x / prop_revenue['Count'].sum())
prop_revenue.head()

,Position,Property,Count,Color,Percent Total Count
0,0,Go,146534,None,0.028244
1,1,Mediterranean Avenue,100800,Dark Purple,0.019429
2,2,Community Chest,105063,None,0.020250
3,3,Baltic Avenue,105504,Dark Purple,0.020335
4,4,Income Tax,114919,None,0.022150


In [20]:
prop_revenue[['Property', 'Count', 'Percent Total Count']].sort_values('Count', ascending=False)

,Property,Count,Percent Total Count
10,Jail,294792,0.056820
25,B&O Railroad,180016,0.034697
0,Go,146534,0.028244
19,New York Avenue,146370,0.028212
5,Reading Railroad,145456,0.028036
35,Short Line,142325,0.027432
17,Community Chest,142079,0.027385
15,Pennsylvania Railroad,141561,0.027285
18,Tennessee Avenue,139961,0.026977
28,Water Works,136470,0.026304


## Determine Single Property Revenue

In [21]:
prop_revenue['Low Rent'] = prop_revenue['Position'].apply(lambda x: properties[x].low_rent)
prop_revenue['High Rent'] = prop_revenue['Position'].apply(lambda x: properties[x].high_rent)

prop_revenue['Low Revenue'] = prop_revenue['Count'] * prop_revenue['Low Rent']
prop_revenue['Percent Total Low Revenue'] = prop_revenue['Low Revenue'].apply(
        lambda x: x / prop_revenue['Low Revenue'].sum()
    )
prop_revenue['High Revenue'] = prop_revenue['Count'] * prop_revenue['High Rent']
prop_revenue['Percent Total High Revenue'] = prop_revenue['High Revenue'].apply(
        lambda x: x / prop_revenue['High Revenue'].sum()
    )


In [22]:
columns = ['Property', 'Count', 'Low Rent', 'Low Revenue', 'Percent Total Low Revenue']
prop_revenue.sort('Low Revenue', ascending=False)[columns]

AttributeError: 'DataFrame' object has no attribute 'sort'

In [24]:
columns = ['Property', 'Count', 'High Rent', 'High Revenue', 'Percent Total High Revenue']
prop_revenue.sort_values('High Revenue', ascending=False)[columns]

,Property,Count,High Rent,High Revenue,Percent Total High Revenue
39,Boardwalk,123400,2000,246800000,0.087937
34,Pennsylvania Avenue,114957,1400,160939800,0.057345
31,Pacific Avenue,121712,1275,155182800,0.055293
32,North Carolina Avenue,119847,1275,152804925,0.054446
37,Park Place,101346,1500,152019000,0.054166
19,New York Avenue,146370,1000,146370000,0.052153
26,Atlantic Avenue,125727,1150,144586050,0.051518
27,Ventnor Avenue,124725,1150,143433750,0.051107
29,Marvin Garden,118636,1200,142363200,0.050726
24,Illinois Avenue,127721,1100,140493100,0.050059


## Group results by color

In [25]:
color_revenue = prop_revenue.groupby( ['Color'] ).sum().reset_index()

In [26]:
color_revenue.head()

,Color,Position,Count,Percent Total Count,Low Rent,High Rent,Low Revenue,Percent Total Low Revenue,High Revenue,Percent Total High Revenue
0,Blue,76,224746,0.043319,85,3500,9717110,0.137923,398819000,0.142103
1,Dark Purple,4,206304,0.039764,6,570,623616,0.008851,58961280,0.021009
2,Green,97,356516,0.068717,80,3950,9499330,0.134832,468927525,0.167084
3,Light Blue,23,342080,0.065934,20,1700,2280222,0.032365,193837550,0.069066
4,None,219,1638771,0.315865,0,0,0,0.000000,0,0.000000


## Total hits per Color

In [28]:
columns = ['Color', 'Count', 'Percent Total Count']
color_revenue.sort_values('Count', ascending=False)[columns]

,Color,Count,Percent Total Count
4,None,1638771,0.315865
7,Railroad,609358,0.117451
5,Orange,420559,0.081061
8,Red,388410,0.074864
10,Yellow,369088,0.071140
6,Pink,368374,0.071002
2,Green,356516,0.068717
3,Light Blue,342080,0.065934
9,Utility,263991,0.050883
0,Blue,224746,0.043319


## Total Revenue per Color

In [30]:
columns = ['Color', 'Count', 'Low Revenue', 'Percent Total Low Revenue']
color_revenue.sort_values('Low Revenue', ascending=False)[columns]

,Color,Count,Low Revenue,Percent Total Low Revenue
7,Railroad,609358,15233950,0.216228
0,Blue,224746,9717110,0.137923
2,Green,356516,9499330,0.134832
10,Yellow,369088,8357208,0.118621
9,Utility,263991,7391748,0.104917
8,Red,388410,7246822,0.102860
5,Orange,420559,6180566,0.087726
6,Pink,368374,3922596,0.055677
3,Light Blue,342080,2280222,0.032365
1,Dark Purple,206304,623616,0.008851


In [31]:
columns = ['Color', 'Count', 'High Revenue', 'Percent Total High Revenue']
color_revenue.sort_values('High Revenue', ascending=False)[columns]

,Color,Count,High Revenue,Percent Total High Revenue
2,Green,356516,468927525,0.167084
10,Yellow,369088,430383000,0.153350
8,Red,388410,414216550,0.147590
5,Orange,420559,406849550,0.144965
0,Blue,224746,398819000,0.142103
6,Pink,368374,294194700,0.104825
3,Light Blue,342080,193837550,0.069066
7,Railroad,609358,121871600,0.043424
1,Dark Purple,206304,58961280,0.021009
9,Utility,263991,18479370,0.006584
